In [11]:
import gym
import pandas as pd
import networkx as nx
import circuitgraph as cg
import torch as tch
from gym import Env, spaces

In [38]:
import numpy as np

In [8]:
%run graph.ipynb

In [39]:
class ChipDesignEnv(Env):
    def __init__(self, circuit_name):
        super(ChipDesignEnv, self).__init__()
        self.circuit_name = circuit_name
        self.circuit_graph = ChipDesignGraph(circuit_name)
        self.grid_size = self.circuit_graph.grid_size
        self.design_graph = self.circuit_graph.design_graph
        self.node_attributes = self.circuit_graph.dict_of_node_to_attributes_
        
        # Observation Space...
        self.action_space = spaces.Discrete(self.grid_size[0] * self.grid_size[1])
        self.observation_space = spaces.Box(low=0, high=1, shape=(self.grid_size[0], self.grid_size[1]), dtype=np.float32)
        
        # Initialize the grid
        self.grid = np.zeros(self.grid_size)
        self.placed_nodes = set()

    def reset(self):
        self.grid = np.zeros(self.grid_size)
        self.placed_nodes = set()
        return self.grid

    def step(self, action):
        node_index = len(self.placed_nodes)
        if node_index >= len(self.node_attributes):
            raise ValueError("All nodes have already been placed.")
        
        node_name = list(self.node_attributes.keys())[node_index]
        node_info = self.node_attributes[node_name]
        
        row, col = divmod(action, self.grid_size[1])
        
        if self.grid[row, col] != 0:
            return self.grid, -1, False, {}
        
        self.grid[row, col] = 1
        self.placed_nodes.add((row, col))
        
        reward = self.calculate_reward()
        done = len(self.placed_nodes) == len(self.node_attributes)
        
        return self.grid, reward, done, {}

    def calculate_reward(self):
        wirelength = self.calculate_wirelength()
        dynamic_power = self.calculate_dynamic_power()
        signal_delay = self.calculate_signal_delay()
        reward = -(wirelength + dynamic_power + signal_delay)
        return reward

    def calculate_wirelength(self):
        wirelength = 0
        positions = list(self.placed_nodes)
        for i in range(len(positions)):
            for j in range(i + 1, len(positions)):
                wirelength += abs(positions[i][0] - positions[j][0]) + abs(positions[i][1] - positions[j][1])
        return wirelength

    def calculate_dynamic_power(self):
        dynamic_power = 0
        positions = list(self.placed_nodes)
        for i in range(len(positions)):
            for j in range(i + 1, len(positions)):
                cap = self.node_attributes[i]["p1_cap"] + self.node_attributes[j]["p2_cap"]
                distance = abs(positions[i][0] - positions[j][0]) + abs(positions[i][1] - positions[j][1])
                dynamic_power += cap * distance
        return dynamic_power

    def calculate_signal_delay(self):
        signal_delay = 0
        positions = list(self.placed_nodes)
        for i in range(len(positions)):
            for j in range(i + 1, len(positions)):
                wire_cap = 1  # Example assumption
                gate_cap = 2  # Example assumption
                distance = abs(positions[i][0] - positions[j][0]) + abs(positions[i][1] - positions[j][1])
                signal_delay += (wire_cap * distance + gate_cap) * distance
        return signal_delay

    def render(self, mode='human'):
        print(self.grid)

In [40]:
env = ChipDesignEnv('c17.v')

In [42]:
state = env.reset()

In [44]:
state

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [45]:
next_state, reward, done, _ = env.step(0)


In [46]:
print("State after placing first node at position 0:")
print(next_state)
print("Reward:", reward)
print("Done:", done)

State after placing first node at position 0:
[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Reward: 0
Done: False


In [47]:
actions = [5, 10, 15, 3, 7]
for action in actions:
    next_state, reward, done, _ = env.step(action)
    print("Next State:")
    print(next_state)
    print("Reward:", reward)
    print("Done:", done)
    if done:
        break

KeyError: 0